In [1]:
import polars as pl
import numpy as np
from pathlib import Path
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from aw_client.client import ActivityWatchClient
from aw_core import Event
from datetime import datetime, timedelta, timezone
from typing import List, Tuple, Optional, Dict, Callable
import pytz
import json

In [2]:
client = ActivityWatchClient("AnatomyOfFlow2")
hostname = "Dominion"
project_dir = Path(globals()['_dh'][0]).parent
start = (datetime.today().astimezone(tz=pytz.timezone("America/Chicago")).replace(
    hour=0, minute=0, second=0, microsecond=0
) - timedelta(days=4)).astimezone(tz=pytz.timezone("UTC"))
end = start + timedelta(days=1)


In [3]:
oura_token=json.load(open(project_dir/'.config'/'oura.json'))['access_token']
from src.oura_handler import get_heartrate_data
oura_data= get_heartrate_data(oura_token, start,end)

In [9]:
print(oura_data)
oura_df=pl.DataFrame(oura_data["data"]).with_columns(pl.col("timestamp").str.to_datetime())# type: ignore
print(oura_df)

{'data': [{'bpm': 92, 'source': 'awake', 'timestamp': '2023-05-04T13:23:49+00:00'}, {'bpm': 78, 'source': 'awake', 'timestamp': '2023-05-04T13:23:51+00:00'}, {'bpm': 76, 'source': 'awake', 'timestamp': '2023-05-04T13:24:20+00:00'}, {'bpm': 76, 'source': 'awake', 'timestamp': '2023-05-04T13:34:00+00:00'}, {'bpm': 79, 'source': 'awake', 'timestamp': '2023-05-04T13:34:07+00:00'}, {'bpm': 93, 'source': 'awake', 'timestamp': '2023-05-04T14:22:24+00:00'}, {'bpm': 101, 'source': 'awake', 'timestamp': '2023-05-04T14:22:39+00:00'}, {'bpm': 106, 'source': 'awake', 'timestamp': '2023-05-04T14:22:40+00:00'}, {'bpm': 95, 'source': 'awake', 'timestamp': '2023-05-04T14:27:28+00:00'}, {'bpm': 94, 'source': 'awake', 'timestamp': '2023-05-04T14:27:57+00:00'}, {'bpm': 97, 'source': 'awake', 'timestamp': '2023-05-04T14:27:59+00:00'}, {'bpm': 98, 'source': 'awake', 'timestamp': '2023-05-04T14:32:40+00:00'}, {'bpm': 100, 'source': 'awake', 'timestamp': '2023-05-04T14:32:46+00:00'}, {'bpm': 103, 'source': 'a

So, I suppose we should define the rules a bit before actually implementing:
1. On a single system, Afk events take priority, followed by window events
2. on multiple systems host-specific priority is determined strictly by time
3. if a higher priority event starts prior to the end of the duration of the current event, then the current event is split into two events
4. if an event has the exact same data as the current event, then the duration of the previous event should be extended
5. if the app associated with a specific event is something which produces events (such as from `"aw-watcher-web-chrome"`), then that window event is replaced by the events of that watcher(alternative would be to "embed" the app events into the window event)



In [4]:
from typing import Callable
from src.aw_merge import bucket_merge


app_map: dict[str, str | Callable[[datetime, datetime], List[Event]]] = {
    "google-chrome": "aw-watcher-web-chrome"
}
events, categories = bucket_merge(
    client, hostname, app_map, start=start, end=start + timedelta(days=1)
)

df = pl.DataFrame(
    {
        "timestamp": [
            e.timestamp for e in events
        ],
        "duration": [e.duration for e in events],
        "end": [(e.timestamp + e.duration) for e in events],
        "data": [str(e.data) for e in events],
        "category": categories,
        "title": [e.data["title"] if "title" in e.data else "afk" for e in events],
    }
)
df

timestamp,duration,end,data,category,title
"datetime[μs, UTC]",duration[μs],"datetime[μs, UTC]",str,str,str
2023-05-04 05:00:00 UTC,8h 7m 2s 502684µs,2023-05-04 13:07:02.502684 UTC,"""{'status': 'af…","""afk""","""afk"""
2023-05-04 13:07:02.502 UTC,11s 443348µs,2023-05-04 13:07:13.945348 UTC,"""{'audible': Fa…","""google-chrome""","""Whyyyyyy in th…"
2023-05-04 13:07:13.971 UTC,1s 880ms,2023-05-04 13:07:15.851 UTC,"""{'audible': Fa…","""google-chrome""","""Whyyyyyy in th…"
2023-05-04 13:07:15.852 UTC,529716µs,2023-05-04 13:07:16.381716 UTC,"""{'audible': Fa…","""google-chrome""","""Reddit - Dive …"
2023-05-04 13:07:16.407 UTC,4s 445ms,2023-05-04 13:07:20.852 UTC,"""{'audible': Fa…","""google-chrome""","""Reddit - Dive …"
2023-05-04 13:07:20.853 UTC,56s 387725µs,2023-05-04 13:08:17.240725 UTC,"""{'audible': Fa…","""google-chrome""","""Whyyyyyy in th…"
2023-05-04 13:08:17.257 UTC,3s 663ms,2023-05-04 13:08:20.920 UTC,"""{'audible': Fa…","""google-chrome""","""Whyyyyyy in th…"
2023-05-04 13:08:20.921 UTC,22s 187629µs,2023-05-04 13:08:43.108629 UTC,"""{'audible': Fa…","""google-chrome""","""BusinessBandic…"
2023-05-04 13:08:43.137 UTC,2s 808ms,2023-05-04 13:08:45.945 UTC,"""{'audible': Fa…","""google-chrome""","""New Tab"""


In [5]:
from polars import DataFrame, Series


localize_col: Callable[[DataFrame,str],Series]=lambda df,col : df.select(
    pl.col(col).cast(pl.Datetime).dt.replace_time_zone("America/New_York")
).to_series()

In [ ]:
merged_time=localize_col(df,"timestamp").append(localize_col(df,"end")).append(localize_col(oura_df,"timestamp")).sort().unique()

In [7]:
aw_fig = px.timeline(
    x_start=localize_col(df,"timestamp"),
    x_end=localize_col(df,"end"),
    y=df.select("category").to_series(),
    category_orders={"category": ["afk", "window", "google-chrome"]},
    color=df.select("category").to_series(),
    text=df.select("title").to_series(),
)
aw_fig.update_traces(textposition="inside")
aw_fig.update_xaxes(fixedrange=False)#,rangeslider_visible=True)
aw_fig.update_yaxes(fixedrange=True,ticklabelposition="inside",categoryorder="array",categoryarray=["google-chrome", "window", "afk"])
aw_fig.show(config={"scrollZoom": True})

In [12]:
fig=make_subplots(specs=[[{"secondary_y": True}]])
for i in range(len(aw_fig.data)):
    fig.add_trace(aw_fig.data[i],secondary_y=False)
fig.update_yaxes(fixedrange=True,ticklabelposition="inside",categoryorder="array",categoryarray=["google-chrome", "window", "afk"])#,autorange=False)
fig.layout.xaxis=aw_fig.layout.xaxis


heart_rate=go.Scatter(x=localize_col(oura_df,"timestamp"),y=oura_df.select('bpm').to_series(),name='Heart Rate',mode='lines',yaxis='y2')
fig.add_trace(heart_rate,secondary_y=True)
#ig.update_layout(autosize=False,height=400)
fig.show()


In [8]:
fig2= make_subplots(specs=[[{"secondary_y": True}]])
for trace in aw_fig.data:
    fig2.add_trace(trace)

heart_rate=go.Scatter(x=localize_col(oura_df,"timestamp"),y=oura_df.select('bpm').to_series(),name='Heart Rate',mode='lines',yaxis='y2')
#fig=make_subplots(shared_xaxis=True,specs=[[{"secondary_y": True}]])
fig2.add_trace(heart_rate,secondary_y=True)


NameError: name 'oura_df' is not defined

In [ ]:
print(aw_fig)

In [ ]:
print(aw_fig.data)
#activity_timeline=fig.data
#fig2 = make_subplots(shared_xaxes=True,specs=[[{"secondary_y": True}]])
#fig2.add_trace(activity_timeline)

#fig2.add_trace(heart_rate,secondary_y=True)
#fig2.show()